# Film Industry Exploratory Analysis

This is an exploratory data analysis project based on the cinema database courtesy of DataCamp. The table schemas we will be primarily using are as follows:

   Films Table        
| Column Name     | Data Type         |
|-----------------|-------------------|
| id              | integer           |
| title           | character varying |
| release_year    | integer           |
| country         | character varying |
| duration        | integer           |
| language        | character varying |
| certification   | character varying |
| gross           | bigint            |
| budget          | bigint            |


   Reviews Table
| Column Name     | Data Type         |
|-----------------|-------------------|
| id              | integer           |
| film_id         | integer           |
| num_user        | integer           |
| num_critic      | integer           |
| imdb_score      | numeric           |
| num_votes       | integer           |
| facebook_likes  | integer           |

## What makes a film "successful"?

First, how can we even quantify film success? A few ways we could do this with our data would be to assess a film's profit generated, IMDb ratings, number of ratings, and facebook likes. Lets do that. 

In [1]:
SELECT
	f.id,
	title,
	imdb_score,
	(gross - budget) / 1000000 AS profit_millions,
	num_votes,
	facebook_likes
FROM cinema.films f
INNER JOIN cinema.reviews r
ON f.id = r.film_id
WHERE gross IS NOT NULL
	AND budget IS NOT NULL
ORDER BY imdb_score DESC;

,id,title,imdb_score,profit_millions,num_votes,facebook_likes
0,742,The Shawshank Redemption,9.3,3,1689764,108000
1,178,The Godfather,9.2,128,1155770,43000
2,3110,The Dark Knight,9.0,348,1676169,37000
3,192,The Godfather: Part II,9.0,44,790926,14000
4,676,Schindler's List,8.9,74,865020,41000
...,...,...,...,...,...,...
3887,1565,Glitter,2.1,-17,19412,0
3888,2946,Who's Your Caddy?,2.0,-1,13815,446
3889,2226,Superbabies: Baby Geniuses 2,1.9,-10,25371,0
3890,2984,Disaster Movie,1.9,-10,74945,0


Here we can easily assess films based off of any of our aforementioned metrics. Sorting by imdb_score, we can see that Fight Club was a box office flop even with its exceptional rating and plenty of voter traffic. Does this make Fight Club a failure, I suppose it depends on who you are and how you are defining "success". Let's identify some films that are perhaps less ambiguous when it comes to perceived success. 

In [2]:
CREATE TEMPORARY TABLE film_metrics AS (
SELECT
	f.id,
	title,
	imdb_score,
	(gross - budget) / 1000000 AS profit_millions,
	num_votes,
	facebook_likes
FROM cinema.films f
INNER JOIN cinema.reviews r
ON f.id = r.film_id
WHERE gross IS NOT NULL
	AND budget IS NOT NULL
ORDER BY imdb_score DESC
);

SELECT *
FROM film_metrics
WHERE imdb_score >= 8
	AND profit_millions >= 100
ORDER BY imdb_score DESC
LIMIT 10;

,id,title,imdb_score,profit_millions,num_votes,facebook_likes
0,178,The Godfather,9.2,128,1155770,43000
1,3110,The Dark Knight,9.0,348,1676169,37000
2,2045,The Lord of the Rings: The Return of the King,8.9,283,1215718,16000
3,3514,Inception,8.8,132,1468200,175000
4,709,Forrest Gump,8.8,274,1251222,59000
5,1667,The Lord of the Rings: The Fellowship of the Ring,8.8,220,1238746,21000
6,276,Star Wars: Episode V - The Empire Strikes Back,8.8,272,837759,17000
7,226,Star Wars: Episode IV - A New Hope,8.7,449,911097,33000
8,201,One Flew Over the Cuckoo's Nest,8.7,107,680041,32000
9,1310,The Matrix,8.7,108,1217752,25000


This is a list of movies that were not only box office hits returning over 100 million dollars in profit, but were also critically praised with ratings above 8. Hard to argue against the success of these films. Now that we have a better idea of how to assess film success, lets explore which kinds of films seem to perform the best.

## Which kinds of films are the most successful?

Within the context of this data, let's see some different ways in which we might want to categorize films.

In [3]:
SELECT 
	country,
	COUNT(*) AS num_films,
	ROUND(AVG(imdb_score), 2) AS avg_rating,
	ROUND(AVG(budget) / 1000000, 1) AS avg_budget_millions,
	ROUND(AVG((gross - budget) / 1000000), 1) AS avg_profit_millions
FROM cinema.films f
INNER JOIN cinema.reviews r
ON f.id = r.film_id
WHERE gross IS NOT NULL
	AND budget IS NOT NULL
GROUP BY country
HAVING COUNT(*) >=10 
ORDER BY avg_budget_millions DESC;

,country,num_films,avg_rating,avg_budget_millions,avg_profit_millions
0,Japan,17,6.96,409.1,-375.0
1,India,14,6.61,112.6,-109.4
2,New Zealand,11,7.48,95.7,22.4
3,China,15,7.07,82.2,-63.8
4,Spain,22,6.97,49.5,-39.3
5,USA,3076,6.39,40.1,17.4
6,Australia,41,6.46,36.1,5.0
7,Germany,82,6.33,35.6,-6.0
8,France,106,6.78,34.0,-15.2
9,UK,325,6.86,30.9,4.9


When grouping by country, we see that Japan seems to have an abnormally high avg budget. This dataset might be storing budget and profit values in terms of a films native currency. When converting from Japanese Yen to USD, we get an average value of about 2.6 million which seems more reasonable. Regardless of which currencies are being displayed, we can see that most of the top countries are actually returning losses on average. Though it might be unfair to judge a nations film industry based on this data given the low sample size of films that are listed for some of these countries. Since the U.S. has a healthy number of films to work with, and we can be more confident of the currency units used, lets focus our analysis further on just American films when exploring which kinds of films are most successful. 

In [4]:
SELECT 
	certification,
	COUNT(*) AS num_films,
	ROUND(AVG(imdb_score), 2) AS avg_rating,
	ROUND(AVG(budget) / 1000000, 1) AS avg_budget_millions,
	ROUND(AVG((gross - budget) / 1000000), 1) AS avg_profit_millions
FROM cinema.films f
INNER JOIN cinema.reviews r
ON f.id = r.film_id
WHERE gross IS NOT NULL
	AND budget IS NOT NULL
	AND country = 'USA'
GROUP BY certification
HAVING COUNT(*) >=10 
ORDER BY avg_budget_millions DESC;

,certification,num_films,avg_rating,avg_budget_millions,avg_profit_millions
0,G,71,6.51,55.4,42.8
1,PG,474,6.24,54.1,27.6
2,PG-13,1117,6.21,53.8,19.5
3,R,1334,6.56,25.1,11.1
4,Unrated,13,7.02,1.8,6.2
5,Not Rated,19,6.82,1.6,3.6
6,,22,6.65,1.3,-0.4


Here we can see a breakdown of some performance metrics by movie certification. It looks as though there is a clear trend with more mature certifications coinciding with lower average profits. If there are stricter criteria for who can see a movie, less tickets may be able to be sold. Let's see how budget might influence films metrics.

In [5]:
WITH budget_thirds AS (
    SELECT
		id,
        budget,
		gross,
        NTILE(3) OVER (ORDER BY budget) AS third
    FROM
        cinema.films
	WHERE budget IS NOT NULL
		AND gross IS NOT NULL
		AND country = 'USA'
)
SELECT
    CASE
        WHEN third = 1 THEN 'Low Budget'
        WHEN third = 2 THEN 'Mid Budget'
        ELSE 'High Budget'
    END AS budget_category,
	ROUND(AVG(imdb_score), 2) AS avg_rating,
	ROUND(AVG(num_votes), 1) AS avg_num_votes,
	ROUND(AVG((gross - budget) / 1000000), 1) AS avg_profit_millions,
	ROUND(SUM(CASE 
			  	WHEN (gross - budget) > 0 THEN 1 
			  	ELSE 0 END) * 100.0 / COUNT(*), 2) AS percentage_profitable
FROM budget_thirds
INNER JOIN cinema.reviews 
ON budget_thirds.id = reviews.film_id
GROUP BY budget_category
ORDER BY percentage_profitable DESC;

,budget_category,avg_rating,avg_num_votes,avg_profit_millions,percentage_profitable
0,Low Budget,6.42,56989.3,15.9,62.48
1,Mid Budget,6.29,92051.7,17.2,57.46
2,High Budget,6.44,174022.1,19.2,52.00


Interesting! The query above first categorizes each American film into equal thirds based on budget, then aggregates our metrics for each third. From this, we can see that while there is little difference in average rating between budget tiers, there are some opposite trends at play in terms of number of votes and percentage of films that return profit, with number of votes going up as budget increases and percentage of profitible films declining. Logically, this makes sense as a higher budget likely includes more marketing resources to drive public engagement. While lower budget films have less of hole to fill when it comes to generating a profit. It's also worth noting that the differences in average profits don't appear all that signficant, since higher budget films inherently have more risk given the larger dollar amounts they need to cover prior to even breaking even. 

## How have films changed over the years?

We've explored different performance metrics and potential categorizations for films, lets now view things from a chronological lense. 

In [6]:
WITH decades AS (
	SELECT
		id AS film_id,
		CONCAT(release_year / 10 * 10, 's') AS decade,
		gross,
		budget
	FROM cinema.films
	WHERE budget IS NOT NULL
		AND gross IS NOT NULL
		AND country = 'USA'
)
SELECT 
	decade,
	COUNT(*) AS num_of_films,
	ROUND(AVG(imdb_score), 2) AS avg_rating,
	ROUND(AVG(num_votes), 1) AS avg_num_votes,
	ROUND(AVG((gross - budget) / 1000000), 1) AS avg_profit_millions,
	ROUND(SUM(CASE 
			  	WHEN (gross - budget) > 0 THEN 1 
			  	ELSE 0 END) * 100.0 / COUNT(*), 2) AS percentage_profitable
FROM decades 
INNER JOIN cinema.reviews
USING(film_id)
GROUP BY decade
ORDER BY decade;

,decade,num_of_films,avg_rating,avg_num_votes,avg_profit_millions,percentage_profitable
0,1920s,2,5.55,2275.5,2.0,100.00
1,1930s,6,8.02,134139.8,66.2,83.33
2,1940s,6,7.52,38473.0,31.0,66.67
3,1950s,6,7.28,49980.0,16.8,100.00
4,1960s,11,7.73,96980.6,53.7,72.73
5,1970s,39,7.40,188178.6,73.2,89.74
6,1980s,178,6.68,96073.7,39.0,79.78
7,1990s,615,6.50,103140.8,18.4,59.51
8,2000s,1366,6.25,99683.0,12.6,52.20
9,2010s,847,6.37,123673.1,16.6,56.91


First, we can now see some more limitations of this dataset as there are less than 10 films listed for per decade prior to the 60s. Also, there is a sharp spike in number of movies included post 1980s. Regardless, there is still some value to be had. It appears as though the 2000s might have been a poor year for the film industry given it's negative average profit calculation. And from the 90s on, there was a sharp decline in the percentage of profitable movies. With the number of films we have to work, let's focus on the 80s to 2010s. 

In [7]:
SELECT
    CONCAT(release_year / 10 * 10, 's') AS decade,
    ROUND(AVG(duration), 2) AS avg_duration_minutes,
    ROUND(COUNT(*) FILTER (WHERE certification = 'G')::numeric / COUNT(*) * 100, 2) AS percent_g,
    ROUND(COUNT(*) FILTER (WHERE certification = 'PG')::numeric / COUNT(*) * 100, 2) AS percent_pg,
	ROUND(COUNT(*) FILTER (WHERE certification = 'PG-13')::numeric / COUNT(*) * 100, 2) AS percent_pg13, 
	ROUND(COUNT(*) FILTER (WHERE certification = 'R')::numeric / COUNT(*) * 100, 2) AS percent_r
FROM cinema.films
WHERE release_year >= 1980
  AND duration IS NOT NULL
  AND certification IS NOT NULL
  AND country = 'USA'
GROUP BY decade
ORDER BY decade;

,decade,avg_duration_minutes,percent_g,percent_pg,percent_pg13,percent_r
0,1980s,109.17,0.44,27.95,10.04,55.02
1,1990s,111.78,2.81,11.72,27.03,57.03
2,2000s,105.45,2.39,14.04,39.59,40.10
3,2010s,105.97,1.09,14.25,36.12,38.57


The table above provides some valuable insights records films from the 80s to the 2010s. First, we can see that there has only been about a 5 minute decline in the average length of movies. We can also see some changes in the prevalences of movie ratings. The percentage of movies that are rated R has declined since the 80s, while we see the opposite for films with a PG-13 rating. 

## Who are the most successful people in the American Film Industry?

Let's introduce two new tables to explore this question. 

   People
| Column    | Type            |
|-----------|-----------------|
| id        | integer         |
| name      | character varying |
| birthdate | date            |
| deathdate | date            |


   Roles
| Column    | Type            |
|-----------|-----------------|
| id        | integer         |
| film_id   | integer         |
| person_id | integer         |
| role      | character       |

In [8]:
WITH film_metrics AS (
	SELECT
		f.id,
		title,
		imdb_score, 
		(gross - budget) / 1000000 AS profit_millions
	FROM cinema.films f
	INNER JOIN cinema.reviews r
	ON f.id = r.film_id
	WHERE budget IS NOT NULL
		AND gross IS NOT NULL 
		AND country = 'USA'
)
SELECT 
	name,
	role,
	COUNT(*) AS num_of_films,
	ROUND(AVG(imdb_score), 2) AS avg_rating,
	RANK() OVER (PARTITION BY role ORDER BY AVG(imdb_score) DESC) AS rating_rank,
	ROUND(AVG(profit_millions), 2) AS avg_profit_mil,
	RANK() OVER (PARTITION BY role ORDER BY AVG(profit_millions) DESC) AS profit_rank
FROM cinema.roles r
INNER JOIN film_metrics fm
	ON r.film_id = fm.id
INNER JOIN cinema.people p
	ON r.person_id = p.id
GROUP BY name, role
HAVING COUNT(*) >= 5
ORDER BY avg_rating DESC;

,name,role,num_of_films,avg_rating,rating_rank,avg_profit_mil,profit_rank
0,Christopher Nolan,director,8,8.43,1,100.63,8
1,Quentin Tarantino,director,8,8.20,2,40.00,29
2,John Ratzenberger,actor,10,7.89,1,113.80,4
3,James Cameron,director,6,7.88,3,194.17,2
4,Peter Jackson,director,6,7.80,4,107.17,7
...,...,...,...,...,...,...,...
604,Jesse McCartney,actor,5,4.82,470,74.40,21
605,Tyler Perry,director,6,4.77,136,41.17,27
606,Brian Levant,director,5,4.76,137,21.60,68
607,Kristin Davis,actor,5,4.70,471,14.00,262


Here we have a nice breakdown of actors and directors along with the average rating and profit for the movies they've worked on, as well as rankings by these values within each role. From this, we can see that Christoper Nolan and Quentin Tarantino appear to be in a class of their own in regard to average IMDb rating. We can also see that some people are very successful both critically and commercially (eg. James Cameron: Ranks 3 and 2), while there are some who are great in one regard, but lacking a bit in the other (eg. Martin Scorsese: Ranks 6 and 117).

## Conclusion

With this data, we were able to assess films based off of a number of metrics (profit generated, IMDb score, number of votes, number of Facebook Likes), and also categorize films into different groupings to see which kinds of films performed the best (country, certification/rating, budget tier). Then we broke down films into decade of release and see how scores, budgets, profits, movie lenghts, and frequency of certifications changed over time. Finally, by utilizing two new tables, we were able to rank both actors and directors by the average scores and profits generated by their films. 